In [1]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

In [3]:
class Event(BaseModel):
    """Informações sobre um evento ocorrido"""
    date: str = Field(description="Data do evento no formato YYYY-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")
    
class EventsList(BaseModel):
    """Lista de Eventos para Extração"""
    events: List[Event] = Field(description="Conjunto de eventos encontrados no texto fornecido")
    
tool_events = convert_to_openai_function(EventsList)
tool_events

{'name': 'EventsList',
 'description': 'Lista de Eventos para Extração',
 'parameters': {'type': 'object',
  'properties': {'events': {'description': 'Conjunto de eventos encontrados no texto fornecido',
    'type': 'array',
    'items': {'description': 'Informações sobre um evento ocorrido',
     'type': 'object',
     'properties': {'date': {'description': 'Data do evento no formato YYYY-MM-DD',
       'type': 'string'},
      'event': {'description': 'Descrição do evento extraído do texto',
       'type': 'string'}},
     'required': ['date', 'event']}}},
  'required': ['events']}}

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as frases de acontecimentos e as extraia integralmente"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_events],
                            function_call={"name":"EventsList"}))

In [5]:
chain.invoke({"input": texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"events":[{"date":"1975-04-04","event":"A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México."},{"date":"1980","event":"Microsoft firmou um contrato com a IBM para fornecer o sistema operacional para o novo PC da IBM, resultando no desenvolvimento do MS-DOS."},{"date":"1985","event":"A Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de sistemas operacionais para desktop."}]}', 'name': 'EventsList'}}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 426, 'total_tokens': 542, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ab5733a3-02a7

In [6]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_events], function_call={'name': 'EventsList'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

{'events': [{'date': '1975-04-04',
   'event': 'A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México.'},
  {'date': '1980',
   'event': 'A Microsoft firmou um contrato com a IBM para fornecer o sistema operacional para o novo PC da IBM, resultando no MS-DOS.'},
  {'date': '1985',
   'event': 'A Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de sistemas operacionais para desktop.'}]}

### Dados da Web

In [7]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://www.techtudo.com.br")
page = loader.load()
page

[Document(page_content=" TechTudo: tecnologia, celular, computador e games                top e últimas       Voltar     top e últimas    últimas do techtudo     notícias     comparativos     dicas e tutoriais     guias     listas     reviews     vídeos     tudo sobre     webstories       celulares e tablets       Voltar     celulares e tablets    notícias de celulares     celulares     tablets     telefonia     smartwatches     e-reader     android     iOS     fórum     --     iPhone 14     iphone 13     galaxy s22 ultra     moto g stylus     redmi note 10       jogos       Voltar     jogos    notícias de jogos     jogos casuais     jogos de esportes     jogos de terror     mais jogos       Voltar     mais jogos    jogos de ação     jogos arcade     jogos de aventura     jogos de corrida     jogos de estratégia     jogos de luta     jogos de raciocínio     jogos de rpg     jogos de tabuleiro     jogos educativos       playstation     xbox     nintendo     fórum     --     gta 5     ro

In [9]:
class BlogPost(BaseModel):
    """Detalhes sobre uma postagem de blog"""
    title: str = Field(description="Título da postagem no blog")
    author: str = Field(description="Nome do autor da postagem no blog")
    
class BlogSite(BaseModel):
    """Conjunto de postagens de blog de um site específico"""
    posts: List[BlogPost] = Field(description="Coleção de postagens de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
tool_blog

{'name': 'BlogSite',
 'description': 'Conjunto de postagens de blog de um site específico',
 'parameters': {'type': 'object',
  'properties': {'posts': {'description': 'Coleção de postagens de blog do site',
    'type': 'array',
    'items': {'description': 'Detalhes sobre uma postagem de blog',
     'type': 'object',
     'properties': {'title': {'description': 'Título da postagem no blog',
       'type': 'string'},
      'author': {'description': 'Nome do autor da postagem no blog',
       'type': 'string'}},
     'required': ['title', 'author']}}},
  'required': ['posts']}}

In [10]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página os posts do blog com as informações especificadas'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))

In [11]:
chain.invoke({"input": page})

[{'title': 'Jogos famosos estão mais baratos neste final de semana; veja lista',
  'author': ''},
 {'title': 'Robô que limpa piscina sozinho pode ser controlado pelo celular',
  'author': ''},
 {'title': 'MODELO GAMER DA XIAOMI Poco X7 Pro: 6 curiosidades do celular que você precisa saber',
  'author': ''},
 {'title': '6 coisas que você precisa ficar de olho na maior feira de tecnologia',
  'author': ''},
 {'title': 'Samsung confirma lançamento do Ballie e apresenta Galaxy Book5',
  'author': ''},
 {'title': 'Nvidia revela RTX 5090 e outras placas de vídeo; preços assustam',
  'author': ''},
 {'title': 'Artbreeder: saiba como usar IA para criar arte e personagens realistas',
  'author': ''},
 {'title': 'Genmoji: como criar emojis no iPhone com o recurso do iOS 18',
  'author': ''},
 {'title': 'Mito ou verdade? 8 fatos sobre micro-ondas que vão te surpreender',
  'author': ''},
 {'title': 'Melhor cooktop 5 bocas: veja modelos para comprar e modernizar a cozinha',
  'author': ''},
 {'tit